# Scenario 1:  Find Number of Records Indexed

## Redis

In [1]:
! redis-cli -p 12000 -3 FT.SEARCH locationidx \* LIMIT 0 0

1# attributes => (empty array)
2# warning => (empty array)
3# total_results => (integer) 1000000
4# format => STRING
5# results => (empty array)


## Pinot

```sql
SELECT COUNT(*) AS Count
FROM locations;
```

In [2]:
! curl -s -X POST -H "Content-Type: application/json" -d @../pinot/s1.json http://localhost:8099/query/sql | jq '.resultTable'

{
  "dataSchema": {
    "columnNames": [
      "Count"
    ],
    "columnDataTypes": [
      "LONG"
    ]
  },
  "rows": [
    [
      1000000
    ]
  ]
}


# Scenario 2:  Display 1 Sample Record

## Redis

In [3]:
! redis-cli -p 12000 -3 FT.SEARCH locationidx \* LIMIT 0 1

1# attributes => (empty array)
2# warning => (empty array)
3# total_results => (integer) 1000000
4# format => STRING
5# results => 
   1) 1# id => "location:30ab1acf-5dfc-4574-9c9d-0f87b372bbd3"
      2# extra_attributes => 1# "$" => "{\"uuid\":\"30ab1acf-5dfc-4574-9c9d-0f87b372bbd3\",\"point_lnglat\":\"-69.79733 43.40754\",\"point_wkt\":\"POINT (-69.79733 43.40754)\",\"dob\":1983}"
      3# values => (empty array)


## Pinot

```sql
SELECT *
FROM locations
LIMIT 1;
```

In [4]:
! curl -s -X POST -H "Content-Type: application/json" -d @../pinot/s2.json http://localhost:8099/query/sql | jq '.resultTable'

{
  "dataSchema": {
    "columnNames": [
      "dob",
      "point_h3",
      "uuid"
    ],
    "columnDataTypes": [
      "LONG",
      "BYTES",
      "STRING"
    ]
  },
  "rows": [
    [
      1995,
      "80c052859e83e425af4043ebaf102363b2",
      "20b82310-7869-49b1-833a-d082f723cc37"
    ]
  ]
}


# Find the UUIDs for the 3 closest users to Woodland Park, CO (-105.0810 38.9922) born in 2010 

## Redis

In [5]:
! redis-cli -p 12000 -3 FT.AGGREGATE locationidx '@dob:[2010 2010]' LOAD 3 @uuid @dob @point_lnglat APPLY 'geodistance(@point_lnglat, -105.0810, 38.9922) * 0.000621371' AS miles SORTBY 2 @miles ASC LIMIT 0 3

1# attributes => (empty array)
2# total_results => (integer) 3
3# format => STRING
4# results => 
   1) 1# extra_attributes => 
         1# "uuid" => "0053c263-898f-4bc8-95a4-9d16e19bf572"
         2# "dob" => "2010"
         3# "point_lnglat" => "-105.12016 39.03974"
         4# "miles" => "3.90092985574"
      2# values => (empty array)
   2) 1# extra_attributes => 
         1# "uuid" => "959dac55-29e1-405e-8a7b-48262d469d62"
         2# "dob" => "2010"
         3# "point_lnglat" => "-105.12903 39.05025"
         4# "miles" => "4.7693952476"
      2# values => (empty array)
   3) 1# extra_attributes => 
         1# "uuid" => "1f351056-30b1-44a0-8cc7-92fa34a43c30"
         2# "dob" => "2010"
         3# "point_lnglat" => "-105.07084 38.90167"
         4# "miles" => "6.2805695196"
      2# values => (empty array)
5# warning => (empty array)


## Pinot

```sql
SELECT uuid,
  dob,
  STDistance(
    point_h3,
    toSphericalGeography(ST_GeomFromText('POINT (-105.0810 38.9922)'))
  ) / 1609.344 AS miles
FROM locations
WHERE (
    dob = 2010
  )
ORDER BY miles ASC
LIMIT 3;
```

In [6]:
! curl -s -X POST -H "Content-Type: application/json" -d @../pinot/s3.json http://localhost:8099/query/sql | jq '.resultTable'

{
  "dataSchema": {
    "columnNames": [
      "uuid",
      "dob",
      "miles"
    ],
    "columnDataTypes": [
      "STRING",
      "LONG",
      "DOUBLE"
    ]
  },
  "rows": [
    [
      "0053c263-898f-4bc8-95a4-9d16e19bf572",
      2010,
      3.899834943756114
    ],
    [
      "959dac55-29e1-405e-8a7b-48262d469d62",
      2010,
      4.76805781165741
    ],
    [
      "1f351056-30b1-44a0-8cc7-92fa34a43c30",
      2010,
      6.278809589188614
    ]
  ]
}


# Find the count of users within the State of Colorado

![Colorado](../data/co.png)

## Redis

In [7]:
! redis-cli -p 12000 -3 FT.SEARCH locationidx '@point_wkt:[WITHIN $Colorado]' PARAMS 2 Colorado 'POLYGON((-109.0448 37.0004,-102.0424 36.9949,-102.0534 41.0006,-109.0489 40.9996,-109.0448 37.0004,-109.0448 37.0004))' LIMIT 0 0 DIALECT 2

1# attributes => (empty array)
2# warning => (empty array)
3# total_results => (integer) 12500
4# format => STRING
5# results => (empty array)


## Pinot

```sql
SELECT COUNT(*) AS CO_Total
FROM locations
WHERE ST_Within(
    point_h3,
    toSphericalGeography(
      ST_GeomFromText(
        'POLYGON((-109.0448 37.0004,-102.0424 36.9949,-102.0534 41.0006,-109.0489 40.9996,-109.0448 37.0004,-109.0448 37.0004))'
      )
    )
  ) IS True;
```

In [8]:
! curl -s -X POST -H "Content-Type: application/json" -d @../pinot/s4.json http://localhost:8099/query/sql | jq '.resultTable'

{
  "dataSchema": {
    "columnNames": [
      "CO_Total"
    ],
    "columnDataTypes": [
      "LONG"
    ]
  },
  "rows": [
    [
      12500
    ]
  ]
}


# Find the count of users by year of birth that are in either of the 2 polygons in the Woodland Park CO area.  Sort by year, ascending.

![WP](../data/wp.png)

## Redis

In [9]:
! redis-cli -p 12000 -3 FT.AGGREGATE locationidx '(@point_wkt:[WITHIN $WP1] | @point_wkt:[WITHIN $WP2])' PARAMS 4 WP1 'POLYGON((-105.080 38.980, -105.020 38.980, -105.020 38.920, -105.080 38.920, -105.080 38.980))' WP2 'POLYGON((-105.110 38.960, -105.050 38.960, -105.050 38.900, -105.110 38.900, -105.110 38.960))' LOAD 1 @dob GROUPBY 1 @dob REDUCE COUNT 0 AS Total SORTBY 2 @Total DESC DIALECT 2

1# attributes => (empty array)
2# total_results => (integer) 14
3# format => STRING
4# results => 
    1) 1# extra_attributes => 
          1# "dob" => "1937"
          2# "Total" => "2"
       2# values => (empty array)
    2) 1# extra_attributes => 
          1# "dob" => "1996"
          2# "Total" => "2"
       2# values => (empty array)
    3) 1# extra_attributes => 
          1# "dob" => "1953"
          2# "Total" => "1"
       2# values => (empty array)
    4) 1# extra_attributes => 
          1# "dob" => "1982"
          2# "Total" => "1"
       2# values => (empty array)
    5) 1# extra_attributes => 
          1# "dob" => "2009"
          2# "Total" => "1"
       2# values => (empty array)
    6) 1# extra_attributes => 
          1# "dob" => "1950"
          2# "Total" => "1"
       2# values => (empty array)
    7) 1# extra_attributes => 
          1# "dob" => "1993"
          2# "Total" => "1"
       2# values => (empty array)
    8) 1# extra_attributes => 
          1# "do

## Pinot

```sql
SELECT dob,
  COUNT(*) AS Total
FROM locations
WHERE ST_Within(
    point_h3,
    toSphericalGeography(
      ST_GeomFromText(
        'POLYGON((-105.080 38.980, -105.020 38.980, -105.020 38.920, -105.080 38.920, -105.080 38.980))'
      )
    )
  ) IS True
  OR ST_Within(
    point_h3,
    toSphericalGeography(
      ST_GeomFromText(
        'POLYGON((-105.110 38.960, -105.050 38.960, -105.050 38.900, -105.110 38.900, -105.110 38.960))'
      )
    )
  ) IS True
GROUP BY dob
ORDER BY Total DESC;
```

In [10]:
! curl -s -X POST -H "Content-Type: application/json" -d @../pinot/s5.json http://localhost:8099/query/sql | jq '.resultTable'

{
  "dataSchema": {
    "columnNames": [
      "dob",
      "Total"
    ],
    "columnDataTypes": [
      "LONG",
      "LONG"
    ]
  },
  "rows": [
    [
      1937,
      2
    ],
    [
      1996,
      2
    ],
    [
      1953,
      1
    ],
    [
      1967,
      1
    ],
    [
      1969,
      1
    ],
    [
      1970,
      1
    ],
    [
      1990,
      1
    ],
    [
      1982,
      1
    ],
    [
      1993,
      1
    ],
    [
      1955,
      1
    ]
  ]
}
